In [1]:
import numpy as np
import os
import xarray as xr




In [2]:
folder_path = '/Users/jan/Programming/first_CNN_on_Radolan/dwd_nc/own_test_data/testdata_two_days_2019_01_01-02.zarr' #'/home/jan/Programming/remote/first_CNN_on_radolan_remote/dwd_nc/own_test_data'
data = 'testdata_two_days_2019_01_01-02.zarr'
load_path = os.path.join(folder_path, data)

In [3]:
print(load_path)

/Users/jan/Programming/first_CNN_on_Radolan/dwd_nc/own_test_data/testdata_two_days_2019_01_01-02.zarr/testdata_two_days_2019_01_01-02.zarr


In [4]:
data = xr.open_dataset(folder_path) # , chunks=None # , chunks=None according to Sebastian more efficient as it avoids dask (default is chunks=1)
# Simply reinstalled environement with numpy pandas xarray zarr jupyter
# Now not the same error as in first_CNN_on_Radolan env for some reason


In [6]:
print(data.dims)

FrozenMappingWarningOnValuesAccess({'step': 1, 'time': 576, 'y': 1200, 'x': 1100})


In [7]:
#I assumed that the data is a xarray dataset with the dimensions time, latitude, longitude
# define block sizes and filter condition here
py, px = 10, 20 # how many pixels in y and x direction
pt = 4 # how many time steps to include in a sample (i.e. context and target)
threshold = 1e-4 # threshold for filter condition
# define slices for the center pixels of the blocks to apply the filter to
center_x = slice(px // 2 - 1, px // 2 + 2) # 3 pixels in x direction
center_y = slice(py // 2 - 1, py // 2 + 2) # 3 pixels in y direction
# partition the data into pt x py x px blocks using coarsen
coarse = data.coarsen(y = py,
                     x = px,
                     time = pt,
                     side = "left", # "left" means that the blocks are aligned to the left of the input
                     boundary="trim") #boundary="trim" removes the last block if it is too small
# construct a new data set, where the blocks are folded into a new dimension
folded = coarse.construct(time = ("time_outer", "time_inner"),
                         y = ("y_outer", "y_inner"),
                         x = ("x_outer", "x_inner"))
# get the value of the center pixels of each block and take their mean
center = folded.isel(y_inner = center_y,
                     x_inner = center_x).mean(("y_inner", "x_inner", "time_inner"))
# define a threshold for the center pixels, i.e. the filter condition
valid_center = center > threshold
# get the outer coordinates for all valid blocks (valid_time, valid_x, valid_y)
valid_coords = np.array(np.nonzero(valid_center.total_precipitation_6hr.values)).T
# For data loading purposes, we can use the valid_coords to get the corresponding blocks
# For example, to get a random block from the valid blocks, we can do the following:
# get a random element from the valid coordinates
sample_coord = valid_coords[np.random.randint(0, valid_coords.shape[0])]
# get the block at the sample coordinate
sample_block = folded.isel(time_outer = sample_coord[0],
                           y_outer = sample_coord[2],
                           x_outer = sample_coord[1])

AttributeError: 'Dataset' object has no attribute 'total_precipitation_6hr'